# 🧪 Notebook de Test Final - Crypto Bot Multi-Exchange

Notebook complet pour tester toutes les fonctionnalités avec Binance, Kraken et Coinbase:
- Tests individuels pour chaque exchange
- Tests de la factory d'exchanges
- Tests du collecteur multi-exchange
- Tests de planification
- Comparaison inter-exchange
- Tests globaux complets

In [1]:
# Configuration du logging pour affichage dans le notebook
import logging
from logger_settings import logger

# Configuration pour afficher les logs dans le notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

logger.info('📋 Configuration du logging terminée')
logger.info('🚀 Notebook de test final prêt - 3 exchanges supportés !')

2026-01-13 09:12:38,381 - logger_settings - INFO - 📋 Configuration du logging terminée
2026-01-13 09:12:38,384 - logger_settings - INFO - 🚀 Notebook de test final prêt - 3 exchanges supportés !


In [2]:
# Import de tous les modules nécessaires
from main import main
from src.collectors.market_collector import MarketCollector
from src.services.exchanges_api.binance_client import BinanceClient
from src.services.exchanges_api.kraken_client import KrakenClient
from src.services.exchanges_api.coinbase_client import CoinbaseClient
from src.scheduler import daily_data_collection, run_once_now
from src.services.exchange_factory import ExchangeFactory, get_exchange_client

## 1️⃣ Tests Individuels des Clients d'Exchange

Test complet de chaque client d'exchange individuellement.

In [3]:
logger.info('🔍 Test du client Binance...')
try:
    binance = BinanceClient()
    logger.info('✅ Client Binance initialisé')
    
    # Test ticker
    ticker = binance.fetch_ticker('BTC/USDT')
    logger.info(f'📊 BTC/USDT @ Binance: {ticker["last"]} USDT')
    
    # Test OHLCV
    ohlcv = binance.fetch_ohlcv('BTC/USDT', '1h', limit=3)
    logger.info(f'📈 {len(ohlcv)} bougies OHLCV Binance')
    
    logger.info('✅ Client Binance testé avec succès')
    
except Exception as e:
    logger.error(f'❌ Erreur client Binance: {e}')

2026-01-13 09:12:39,058 - logger_settings - INFO - 🔍 Test du client Binance...
2026-01-13 09:12:39,354 - logger_settings - INFO - Synchro de l'heure Binance réussie
2026-01-13 09:12:39,601 - logger_settings - INFO - Initialisation de l'exchange réussie
2026-01-13 09:12:39,603 - logger_settings - INFO - ✅ Client Binance initialisé
2026-01-13 09:12:43,596 - logger_settings - INFO - 📊 BTC/USDT @ Binance: 91934.24 USDT
2026-01-13 09:12:43,901 - logger_settings - INFO - 📈 3 bougies OHLCV Binance
2026-01-13 09:12:43,901 - logger_settings - INFO - ✅ Client Binance testé avec succès


In [4]:
logger.info('🔍 Test du client Kraken...')
try:
    kraken = KrakenClient(use_auth=False)
    logger.info('✅ Client Kraken initialisé')
    
    # Test ticker
    ticker = kraken.fetch_ticker('BTC/USD')
    logger.info(f'📊 BTC/USD @ Kraken: {ticker["last"]} USD')
    
    # Test OHLCV
    ohlcv = kraken.fetch_ohlcv('BTC/USD', '1h', limit=3)
    logger.info(f'📈 {len(ohlcv)} bougies OHLCV Kraken')
    
    logger.info('✅ Client Kraken testé avec succès')
    
except Exception as e:
    logger.error(f'❌ Erreur client Kraken: {e}')

2026-01-13 09:12:43,913 - logger_settings - INFO - 🔍 Test du client Kraken...
2026-01-13 09:12:44,006 - logger_settings - INFO - Synchro de l'heure Kraken réussie
2026-01-13 09:12:45,033 - logger_settings - INFO - Initialisation de l'exchange Kraken réussie
2026-01-13 09:12:45,034 - logger_settings - INFO - ✅ Client Kraken initialisé
2026-01-13 09:12:48,034 - logger_settings - INFO - 📊 BTC/USD @ Kraken: 91872.9 USD
2026-01-13 09:12:49,302 - logger_settings - INFO - 📈 3 bougies OHLCV Kraken
2026-01-13 09:12:49,303 - logger_settings - INFO - ✅ Client Kraken testé avec succès


In [5]:
logger.info('🔍 Test du client Coinbase...')
try:
    coinbase = CoinbaseClient(use_auth=False)
    logger.info('✅ Client Coinbase initialisé')
    
    # Test ticker
    ticker = coinbase.fetch_ticker('BTC/USD')
    logger.info(f'📊 BTC/USD @ Coinbase: {ticker["last"]} USD')
    
    # Test OHLCV
    ohlcv = coinbase.fetch_ohlcv('BTC/USD', '1h', limit=3)
    logger.info(f'📈 {len(ohlcv)} bougies OHLCV Coinbase')
    
    logger.info('✅ Client Coinbase testé avec succès')
    
except Exception as e:
    logger.error(f'❌ Erreur client Coinbase: {e}')

2026-01-13 09:12:49,316 - logger_settings - INFO - 🔍 Test du client Coinbase...
2026-01-13 09:12:49,480 - logger_settings - INFO - Synchronisation de l'heure Coinbase réussie
2026-01-13 09:12:51,296 - logger_settings - INFO - Initialisation de l'exchange Coinbase réussie
2026-01-13 09:12:51,297 - logger_settings - INFO - ✅ Client Coinbase initialisé
2026-01-13 09:12:51,322 - logger_settings - INFO - 📊 BTC/USD @ Coinbase: 91845.09 USD
2026-01-13 09:12:51,889 - logger_settings - INFO - 📈 3 bougies OHLCV Coinbase
2026-01-13 09:12:51,889 - logger_settings - INFO - ✅ Client Coinbase testé avec succès


## 2️⃣ Tests de la Factory d'Exchange

Test de la création unifiée de clients via ExchangeFactory.

In [6]:
logger.info('🔍 Test de la factory d\'exchanges...')
try:
    # Créer clients pour tous les exchanges
    binance_client = ExchangeFactory.create_exchange('binance')
    kraken_client = ExchangeFactory.create_exchange('kraken')
    coinbase_client = ExchangeFactory.create_exchange('coinbase')
    logger.info('✅ Tous les clients créés via factory')
    
    # Test fonction utilitaire
    binance_client_2 = get_exchange_client('binance')
    kraken_client_2 = get_exchange_client('kraken')
    coinbase_client_2 = get_exchange_client('coinbase')
    logger.info('✅ Fonction utilitaire testée')
    
    # Test exchange non supporté
    try:
        ExchangeFactory.create_exchange('ftx')
    except ValueError:
        logger.info('✅ Validation des exchanges fonctionnelle')
    
    # Liste des exchanges supportés
    supported = ExchangeFactory.get_supported_exchanges()
    logger.info(f'📋 Exchanges supportés: {supported}')
    
    logger.info('✅ factory d\'exchanges testée avec succès')
    
except Exception as e:
    logger.error(f'❌ Erreur factory d\'exchanges: {e}')

2026-01-13 09:12:51,898 - logger_settings - INFO - 🔍 Test de la factory d'exchanges...
2026-01-13 09:12:51,899 - logger_settings - INFO - Création du client Binance
2026-01-13 09:12:52,195 - logger_settings - INFO - Synchro de l'heure Binance réussie
2026-01-13 09:12:52,430 - logger_settings - INFO - Initialisation de l'exchange réussie
2026-01-13 09:12:52,431 - logger_settings - INFO - Création du client Kraken
2026-01-13 09:12:52,513 - logger_settings - INFO - Synchro de l'heure Kraken réussie
2026-01-13 09:12:53,491 - logger_settings - INFO - Initialisation de l'exchange Kraken réussie
2026-01-13 09:12:53,492 - logger_settings - INFO - Création du client Coinbase
2026-01-13 09:12:53,643 - logger_settings - INFO - Synchronisation de l'heure Coinbase réussie
2026-01-13 09:12:56,191 - logger_settings - INFO - Initialisation de l'exchange Coinbase réussie
2026-01-13 09:12:56,193 - logger_settings - INFO - ✅ Tous les clients créés via factory
2026-01-13 09:12:56,193 - logger_settings - I

## 3️⃣ Tests du Collecteur Multi-Exchange

Test du MarketCollector avec les trois exchanges.

In [7]:
logger.info('🔍 Test MarketCollector avec Binance...')
try:
    binance_pairs = ['BTC/USDT', 'ETH/USDT']
    collector = MarketCollector(binance_pairs, ['1h'], 'binance')
    collector.fetch_and_store()
    logger.info('✅ MarketCollector Binance testé')
    
except Exception as e:
    logger.error(f'❌ Erreur MarketCollector Binance: {e}')

2026-01-13 09:13:00,296 - logger_settings - INFO - 🔍 Test MarketCollector avec Binance...


2026-01-13 09:13:00,576 - logger_settings - INFO - Synchro de l'heure Binance réussie
2026-01-13 09:13:00,814 - logger_settings - INFO - Initialisation de l'exchange réussie
2026-01-13 09:13:04,856 - logger_settings - INFO - ✅ BTC/USDT 1h sauvegardé
2026-01-13 09:13:05,222 - logger_settings - INFO - ✅ ETH/USDT 1h sauvegardé
2026-01-13 09:13:05,222 - logger_settings - INFO - ✅ MarketCollector Binance testé


In [8]:
logger.info('🔍 Test MarketCollector avec Kraken...')
try:
    kraken_pairs = ['BTC/USD', 'ETH/USD']
    collector = MarketCollector(kraken_pairs, ['1h'], 'kraken')
    collector.fetch_and_store()
    logger.info('✅ MarketCollector Kraken testé')
    
except Exception as e:
    logger.error(f'❌ Erreur MarketCollector Kraken: {e}')

2026-01-13 09:13:05,235 - logger_settings - INFO - 🔍 Test MarketCollector avec Kraken...
2026-01-13 09:13:05,316 - logger_settings - INFO - Synchro de l'heure Kraken réussie
2026-01-13 09:13:06,372 - logger_settings - INFO - Initialisation de l'exchange Kraken réussie
2026-01-13 09:13:09,595 - logger_settings - INFO - ✅ BTC/USD 1h sauvegardé
2026-01-13 09:13:10,735 - logger_settings - INFO - ✅ ETH/USD 1h sauvegardé
2026-01-13 09:13:10,735 - logger_settings - INFO - ✅ MarketCollector Kraken testé


In [9]:
logger.info('🔍 Test MarketCollector avec Coinbase...')
try:
    coinbase_pairs = ['BTC/USD', 'ETH/USD']
    collector = MarketCollector(coinbase_pairs, ['1h'], 'coinbase')
    collector.fetch_and_store()
    logger.info('✅ MarketCollector Coinbase testé')
    
except Exception as e:
    logger.error(f'❌ Erreur MarketCollector Coinbase: {e}')

2026-01-13 09:13:10,743 - logger_settings - INFO - 🔍 Test MarketCollector avec Coinbase...
2026-01-13 09:13:10,904 - logger_settings - INFO - Synchronisation de l'heure Coinbase réussie
2026-01-13 09:13:12,851 - logger_settings - INFO - Initialisation de l'exchange Coinbase réussie
2026-01-13 09:13:13,305 - logger_settings - INFO - ✅ BTC/USD 1h sauvegardé
2026-01-13 09:13:13,516 - logger_settings - INFO - ✅ ETH/USD 1h sauvegardé
2026-01-13 09:13:13,517 - logger_settings - INFO - ✅ MarketCollector Coinbase testé


## 4️⃣ Tests des Fonctions de Planification

Test des fonctions de scheduling avec les trois exchanges.

In [10]:
logger.info('🔍 Test des fonctions de planification multi-exchange...')
try:
    # Test avec Binance
    logger.info('Test daily_data_collection Binance...')
    daily_data_collection(['BTC/USDT'], ['1h'], 'binance')
    logger.info('✅ Binance OK')
    
    # Test avec Kraken
    logger.info('Test daily_data_collection Kraken...')
    daily_data_collection(['BTC/USD'], ['1h'], 'kraken')
    logger.info('✅ Kraken OK')
    
    # Test avec Coinbase
    logger.info('Test daily_data_collection Coinbase...')
    daily_data_collection(['BTC/USD'], ['1h'], 'coinbase')
    logger.info('✅ Coinbase OK')
    
    # Test run_once_now avec Coinbase
    logger.info('Test run_once_now Coinbase...')
    run_once_now(['ETH/USD'], ['6h'], 'coinbase')
    logger.info('✅ run_once_now OK')
    
    logger.info('✅ Toutes les fonctions de planification testées')
    
except Exception as e:
    logger.error(f'❌ Erreur fonctions de planification: {e}')

2026-01-13 09:13:13,530 - logger_settings - INFO - 🔍 Test des fonctions de planification multi-exchange...
2026-01-13 09:13:13,531 - logger_settings - INFO - Test daily_data_collection Binance...
2026-01-13 09:13:13,532 - logger_settings - INFO - Début de la collecte quotidienne de données (binance)
2026-01-13 09:13:13,533 - logger_settings - INFO - Timeframes normalisés: ['1h']
2026-01-13 09:13:13,811 - logger_settings - INFO - Synchro de l'heure Binance réussie
2026-01-13 09:13:14,045 - logger_settings - INFO - Initialisation de l'exchange réussie
2026-01-13 09:13:18,336 - logger_settings - INFO - ✅ BTC/USDT 1h sauvegardé
2026-01-13 09:13:18,339 - logger_settings - INFO - ✅ Collecte quotidienne binance terminée avec succès
2026-01-13 09:13:18,374 - logger_settings - INFO - ✅ Binance OK
2026-01-13 09:13:18,374 - logger_settings - INFO - Test daily_data_collection Kraken...
2026-01-13 09:13:18,375 - logger_settings - INFO - Début de la collecte quotidienne de données (kraken)
2026-01-1

## 5️⃣ Comparaison Inter-Exchange

Comparaison des prix entre les trois exchanges et analyse des opportunités.

In [11]:
logger.info('🔍 Comparaison des trois exchanges...')
try:
    # Récupération des prix
    binance = BinanceClient()
    kraken = KrakenClient(use_auth=False)
    coinbase = CoinbaseClient(use_auth=False)
    
    # Prix BTC (note: Binance utilise USDT, les autres USD)
    btc_binance = binance.fetch_ticker('BTC/USDT')['last']
    btc_kraken = kraken.fetch_ticker('BTC/USD')['last']
    btc_coinbase = coinbase.fetch_ticker('BTC/USD')['last']
    
    logger.info(f'📊 BTC - Binance: {btc_binance} USDT')
    logger.info(f'📊 BTC - Kraken: {btc_kraken} USD')
    logger.info(f'📊 BTC - Coinbase: {btc_coinbase} USD')
    
    # Calcul des différences (en USD pour comparaison)
    prices = {
        'Binance': btc_binance,  # USDT ≈ USD
        'Kraken': btc_kraken,
        'Coinbase': btc_coinbase
    }
    
    # Trouver min et max
    min_price = min(prices.values())
    max_price = max(prices.values())
    spread = max_price - min_price
    spread_percent = (spread / min_price) * 100
    
    logger.info(f'📏 Spread: {spread:.2f} USD ({spread_percent:.2f}%)')
    
    # Analyse des opportunités
    if spread_percent > 1:
        best_buy = min(prices, key=prices.get)
        best_sell = max(prices, key=prices.get)
        logger.warning(f'🔥 Opportunité d\'arbitrage: {best_buy} → {best_sell} = {spread_percent:.2f}%')
    else:
        logger.info('📊 Prix alignés entre exchanges (< 1% de différence)')
    
    logger.info('✅ Comparaison multi-exchange terminée')
    
except Exception as e:
    logger.error(f'❌ Erreur comparaison: {e}')

2026-01-13 09:13:27,755 - logger_settings - INFO - 🔍 Comparaison des trois exchanges...
2026-01-13 09:13:28,034 - logger_settings - INFO - Synchro de l'heure Binance réussie
2026-01-13 09:13:28,278 - logger_settings - INFO - Initialisation de l'exchange réussie
2026-01-13 09:13:28,447 - logger_settings - INFO - Synchro de l'heure Kraken réussie
2026-01-13 09:13:29,380 - logger_settings - INFO - Initialisation de l'exchange Kraken réussie
2026-01-13 09:13:29,558 - logger_settings - INFO - Synchronisation de l'heure Coinbase réussie
2026-01-13 09:13:31,244 - logger_settings - INFO - Initialisation de l'exchange Coinbase réussie
2026-01-13 09:13:37,971 - logger_settings - INFO - 📊 BTC - Binance: 91962.08 USDT
2026-01-13 09:13:37,973 - logger_settings - INFO - 📊 BTC - Kraken: 91850.5 USD
2026-01-13 09:13:37,973 - logger_settings - INFO - 📊 BTC - Coinbase: 91879.25 USD
2026-01-13 09:13:37,974 - logger_settings - INFO - 📏 Spread: 111.58 USD (0.12%)
2026-01-13 09:13:37,975 - logger_settings -

## 6️⃣ Tests Globaux Complets

Exécution complète avec tous les exchanges et tests finaux.

In [12]:
logger.info('Tests globaux complets...')
try:
    # Test complet pour chaque exchange
    exchanges = ['binance', 'kraken', 'coinbase']
    pairs_map = {
        'binance': ['BTC/USDT', 'ETH/USDT'],
        'kraken': ['BTC/USD', 'ETH/USD'],
        'coinbase': ['BTC/USD', 'ETH/USD']
    }
    
    for exchange in exchanges:
        logger.info(f'Test complet {exchange.capitalize()}...')
        collector = MarketCollector(pairs_map[exchange], ['1h'], exchange)
        collector.fetch_and_store()
        logger.info(f'✅ {exchange.capitalize()} terminé')
    
    logger.info('Tous les tests globaux terminés avec succès !')
    logger.info('Système multi-exchange pleinement opérationnel')
    
except Exception as e:
    logger.error(f'💥 Erreur tests globaux: {e}')

2026-01-13 09:13:37,988 - logger_settings - INFO - Tests globaux complets...
2026-01-13 09:13:37,990 - logger_settings - INFO - Test complet Binance...
2026-01-13 09:13:38,378 - logger_settings - INFO - Synchro de l'heure Binance réussie
2026-01-13 09:13:38,617 - logger_settings - INFO - Initialisation de l'exchange réussie
2026-01-13 09:13:42,197 - logger_settings - INFO - ✅ BTC/USDT 1h sauvegardé
2026-01-13 09:13:42,455 - logger_settings - INFO - ✅ ETH/USDT 1h sauvegardé
2026-01-13 09:13:42,456 - logger_settings - INFO - ✅ Binance terminé
2026-01-13 09:13:42,456 - logger_settings - INFO - Test complet Kraken...
2026-01-13 09:13:42,544 - logger_settings - INFO - Synchro de l'heure Kraken réussie
2026-01-13 09:13:43,523 - logger_settings - INFO - Initialisation de l'exchange Kraken réussie
2026-01-13 09:13:46,755 - logger_settings - INFO - ✅ BTC/USD 1h sauvegardé
2026-01-13 09:13:47,953 - logger_settings - INFO - ✅ ETH/USD 1h sauvegardé
2026-01-13 09:13:47,955 - logger_settings - INFO 

## 📖 Guide d'Utilisation en Ligne de Commande

### Commandes disponibles:

```bash
# Exécution unique
python main.py                    # Binance (défaut)
python main.py --exchange kraken   # Kraken
python main.py --exchange coinbase # Coinbase

# Planification quotidienne
python main.py --schedule                    # Binance
python main.py --schedule --exchange kraken   # Kraken
python main.py --schedule --exchange coinbase # Coinbase

# Aide
python main.py --help
```

### Paires par Exchange:

**Binance:** BTC/USDT, ETH/USDT, BNB/USDT, etc. (format USDT)
**Kraken:** BTC/USD, ETH/USD, XRP/USD, etc. (format USD)
**Coinbase:** BTC/USD, ETH/USD, LTC/USD, etc. (format USD)

⚠️ **Important:** Adaptez les paires à l'exchange utilisé

### Spécificités:

- **Binance:** Nécessite des clés API même pour les données publiques
- **Kraken:** Pas besoin de clés API pour les données de marché
- **Coinbase:** Pas besoin de clés API pour les données de marché
